In [3]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

**Deploy hyperparameters**

In [2]:
input_size = 784       # The image size = 28 x 28 = 784
hidden_size = 500      # The number of nodes at the hidden layer
num_classes = 10       # The number of output classes. In this case, from 0 to 9
num_epochs = 5         # The number of times entire dataset is trained
batch_size = 100       # The size of input data took for one iteration
learning_rate = 0.001 # The speed of convergence

**Download data set**

In [3]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
transform=transforms.ToTensor())

Processing...
Done!


**Load data set**

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
shuffle=False)

**Building the FNN (The FNN includes two fully-connected layers (i.e. fc1 & fc2) and a non-linear ReLU layer in between)**

In [4]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    

**Instantiate the network**

In [9]:
net = Net(input_size, hidden_size, num_classes)

**Selecting the Loss Function and Optimizer**

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

**Training the model**

In [14]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3439
Epoch [1/5], Step [200/600], Loss: 0.1659
Epoch [1/5], Step [300/600], Loss: 0.0830
Epoch [1/5], Step [400/600], Loss: 0.2091
Epoch [1/5], Step [500/600], Loss: 0.1569
Epoch [1/5], Step [600/600], Loss: 0.0748
Epoch [2/5], Step [100/600], Loss: 0.3648
Epoch [2/5], Step [200/600], Loss: 0.0664
Epoch [2/5], Step [300/600], Loss: 0.0466
Epoch [2/5], Step [400/600], Loss: 0.0566
Epoch [2/5], Step [500/600], Loss: 0.0985
Epoch [2/5], Step [600/600], Loss: 0.0747
Epoch [3/5], Step [100/600], Loss: 0.0860
Epoch [3/5], Step [200/600], Loss: 0.1044
Epoch [3/5], Step [300/600], Loss: 0.1204
Epoch [3/5], Step [400/600], Loss: 0.0710
Epoch [3/5], Step [500/600], Loss: 0.0732
Epoch [3/5], Step [600/600], Loss: 0.0560
Epoch [4/5], Step [100/600], Loss: 0.0736
Epoch [4/5], Step [200/600], Loss: 0.1115
Epoch [4/5], Step [300/600], Loss: 0.0119
Epoch [4/5], Step [400/600], Loss: 0.0221
Epoch [4/5], Step [500/600], Loss: 0.0253
Epoch [4/5], Step [600/600], Loss:

**Got an error..apparently a side effect of Variable/Tensor merge. Solution was to change "loss.data[0] to loss.item()_"**

**Let's test the model**

In [15]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 98 %


Visualise loss and filter

In [16]:
import matplotlib.pyplot as plt